In [2]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_case as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.CLIPS


In [5]:
models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [6]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

model_phases = dr.CLIPS

label_type = dr.SelfReports.AROUSAL
threshold = "fixed"

x, y = train.Train_CASE.get_case_data(
    metrics, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    # features = {name: metrics for name in models.keys()}
    features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.71445998 0.71546328 0.71451724 0.7132386  0.7042551  0.71800536
 0.74564859 0.73259122 0.72427387 0.71502941 0.71173207 0.73161434
 0.72310993 0.72585083 0.7221941  0.71739509 0.73598554 0.71958829
 0.71175038 0.70849077 0.72633218 0.74197224 0.72782614 0.71887109
 0.70984831        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [29 16], Predictions: [0], [45]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [29 16], Predictions: [0 1], [36  9]
Training RF ...
Model RF, Actual: [0 1], [29 16], Predictions: [0 1], [36  9]
Training XGB ...
Model XGB, Actual: [0 1], [29 16], Predictions: [0 1], [36  9]

None


[('rmssd', 111), ('sdnn', 106), ('hf_rr', 98), ('lf_rr', 88), ('bpm', 67)]


[('bpm'

One or more of the test scores are non-finite: [0.70387006 0.68118279 0.69464611 0.68603544 0.6959918  0.6936287
 0.70019918 0.7081357  0.70248303 0.70566534 0.69525403 0.6929654
 0.7077659  0.69635876 0.70330711 0.69592938 0.70118802 0.69972111
 0.69785478 0.70921823 0.68700461 0.70676679 0.71204625 0.70581384
 0.71380276        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [31 14], Predictions: [0 1], [44  1]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [31 14], Predictions: [0 1], [35 10]
Training RF ...
Model RF, Actual: [0 1], [31 14], Predictions: [0 1], [35 10]
Training XGB ...
Model XGB, Actual: [0 1], [31 14], Predictions: [0 1], [36  9]

None


[('hf_rr', 126), ('rmssd', 123), ('lf_rr', 104), ('bpm', 78), ('sdnn', 37)

...

Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.001, 'kernel': 'sigmoid'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 7, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [35 10], Predictions: [0], [45]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [35 10], Predictions: [0 1], [40  5]
Training RF ...
Model RF, Actual: [0 1], [35 10], Predictions: [0 1], [39  6]
Training XGB ...
Model XGB, Actual: [0 1], [35 10], Predictions: [0 1], [40  5]

None


[('rmssd', 112), ('hf_rr', 97), ('lf_rr', 92), ('bpm', 87), ('sdnn', 51)]


[('

One or more of the test scores are non-finite: [0.72801931 0.74395976 0.73532766 0.7152895  0.7186667  0.69164855
 0.72909616 0.72140142 0.71709594 0.72317042 0.68526685 0.70173337
 0.70487784 0.7079379  0.71545413 0.6982833  0.71125861 0.71570675
 0.71729564 0.7200529  0.68550639 0.71435775 0.71872428 0.71752365
 0.71189794        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.01, 'kernel': 'sigmoid'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 20, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [33 12], Predictions: [0], [45]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [33 12], Predictions: [0 1], [33 12]
Training RF ...
Model RF, Actual: [0 1], [33 12], Predictions: [0 1], [34 11]
Training XGB ...
Model XGB, Actual: [0 1], [33 12], Predictions: [0 1], [36  9]

None


[('hf_rr', 128), ('sdnn', 117), ('rmssd', 107), ('lf_rr', 68), ('bpm', 61)]


[

One or more of the test scores are non-finite: [0.68272397 0.69067553 0.6938461  0.70068321 0.71343466 0.69373303
 0.68887817 0.7043462  0.72012044 0.72461906 0.70943801 0.71986378
 0.71612604 0.72185272 0.72226776 0.69654294 0.7018432  0.69947495
 0.69946208 0.70397906 0.71225256 0.71643725 0.70857324 0.7098591
 0.71086422        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [33 12], Predictions: [0], [45]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [33 12], Predictions: [0 1], [33 12]
Training RF ...
Model RF, Actual: [0 1], [33 12], Predictions: [0 1], [31 14]
Training XGB ...
Model XGB, Actual: [0 1], [33 12], Predictions: [0 1], [33 12]

None


[('lf_rr', 112), ('hf_rr', 100), ('rmssd', 90), ('bpm', 85), ('sdnn', 74)]


[('hf